In [41]:
import pandas as pd
import os
import logging

In [42]:
logging.basicConfig(level=logging.INFO, format=' %(asctime)s - %(levelname)s - %(message)s')

In [43]:
baojiadan_folder = 'D:\\Program Files (x86)\\百度云同步盘\\Dropbox\\-E·J- 2014.5.1\\2015.9.26 燕文\\'

In [44]:
baojiadan_file_list = list()

In [45]:
for file_name in os.listdir(baojiadan_folder):
    if file_name.startswith('广州燕文WISH报价单'):
        baojiadan_file_list.append(file_name)

In [46]:
latest_baojiadan_file_name = baojiadan_file_list[-1]
# latest_baojiadan_file_name

In [47]:
latest_baojiadan_path = os.path.join(baojiadan_folder, latest_baojiadan_file_name)
# latest_baojiadan_path

In [48]:
baojiadan_sheets_dict = pd.read_excel(latest_baojiadan_path, sheetname=None)
# type(baojiadan)

In [49]:
wanted_shipping_service_list = ['Wish燕文航空经济小包-普货',
                                'Wish燕文航空经济小包-特货',
                                'Wish燕文专线平邮小包-普货',
                                'Wish燕文专线平邮小包-特货',
                                'WISH燕文C平邮小包',
                                'Wish燕邮宝平邮-特货',
                                'Wish燕文航空挂号小包-普货',
                                'Wish燕文航空挂号小包-特货',
                                'WISH燕文C挂号小包',
                                'WISH燕文C优先小包',
                                'Wish燕邮宝挂号-特货',
                                'Wish燕文航空易派小包-普货',
                                'Wish燕文航空易派小包-特货',
                                'Wish燕文专线追踪小包-普货',
                                'Wish燕文专线追踪小包-特货',
                                'WISH燕特快-普货',
                                'WISH燕特快-特货'
]

In [50]:
export_path = os.path.join(baojiadan_folder, '报价单','燕文（wish）报价表格.xlsx')
writer = pd.ExcelWriter(export_path)

for service_name in wanted_shipping_service_list:
    if service_name in baojiadan_sheets_dict.keys():
        logging.info(service_name + ' is ready to export.')
        df = baojiadan_sheets_dict[service_name]
        try:
            for c in df.columns:
                for index, r in enumerate(df[c]):
                    if r == '国家':
                        country_cell_location = [index, c]
#                         print(country_cell_location)
                    if r == '价格使用说明:':
                        end_location = [index, c]
#                         print(country_cell_location)
                        break
            df = df[country_cell_location[0]+1:end_location[0]]
            df = df.dropna(how='all')
            df = df[df[country_cell_location[1]].str.len() < 10]
#             df['service_name'] = service_name
            print(df.head(5))
            
#             tiers = [1, 2, 3]
#             for tier in tiers:
#                 this_c_name = 'tier_' + str(tier)
#                 next_c = tier + 1
#                 try:
#                     df[this_c_name + '_base'] = df[[tier]]
#                     df[this_c_name + '_surcharge'] = df[[next_c]]
#                     df[this_c_name + '_total'] = df[[tier]].astype(float) * 0.35 + df[this_c_name + '_surcharge'].astype(float)
#                 except Exception as err:
#                     print(str(err))
            
            kg = 0.35
            tiers = [1, 2, 3]
            
            if service_name == 'Wish燕文航空易派小包-普货':
                tiers = [1]
            
            for tier in tiers:
                this_c_name = 'tier_' + str(tier)
                column_num = 2 * tier - 1
                try:
                    df[this_c_name + '_kg_fee'] = df[[column_num]] * kg
                    df[this_c_name + '_kg_fee'] = df[this_c_name + '_kg_fee'].astype(float)
                    df[this_c_name + '_surcharge'] = df[[column_num + 1]].astype(float)
                    df[this_c_name + '_total'] = df[this_c_name + '_kg_fee'] + df[this_c_name + '_surcharge']
                except Exception as err:
                    print(str(err))
                    
#             df_for_merge = df[['service_name', 0, 'tier_1_total']]
            
#             pd_to_merge.append(df_for_merge)
    
            df.to_excel(writer, sheet_name=service_name, index=False)
            logging.info(service_name + ' exported.')
            # df.to_csv(os.path.join(baojiadan_folder, '报价单', key + '.csv'))
        except Exception as err:
            logging.info(service_name + ' failed to export. ' + str(err))
            pass
        # df.to_csv(os.path.join(baojiadan_folder, '报价单', key + '.xlsx'))
        
writer.save()

 2020-03-19 00:20:04,219 - INFO - Wish燕文航空经济小包-普货 is ready to export.
 2020-03-19 00:20:04,396 - INFO - Wish燕文航空经济小包-普货 exported.
 2020-03-19 00:20:04,397 - INFO - Wish燕文航空经济小包-特货 is ready to export.
 2020-03-19 00:20:04,505 - INFO - Wish燕文航空经济小包-特货 exported.
 2020-03-19 00:20:04,506 - INFO - Wish燕文专线平邮小包-普货 is ready to export.
 2020-03-19 00:20:04,530 - INFO - Wish燕文专线平邮小包-普货 exported.
 2020-03-19 00:20:04,531 - INFO - Wish燕文专线平邮小包-特货 is ready to export.


  Wish燕文航空经济小包-普货 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4  返回主目录
4              美国         78        6.5         57        6.3    NaN
5             加拿大         80        6.3         58        6.8    NaN
6              英国         78        6.3         70          5    NaN
7              法国         70        6.5         56        6.8    NaN
8              德国         75        6.5         53          7    NaN
  Wish燕文航空经济小包-特货 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4  返回主目录
4              美国         78        5.8         56        6.5    NaN
5             加拿大         80        6.3        128       0.45    NaN
6              英国         78        6.5         70        5.5    NaN
7              法国         69        6.5         70        5.5    NaN
8              德国         74        6.5         52          7    NaN
  Wish燕文专线平邮小包-普货 Unnamed: 1 Unnamed: 2      Unnamed: 3 Unnamed: 4  返回主目录
3              英国       56.5        8.9  0.001-0.149 KG      0.001    NaN
4              英国       

 2020-03-19 00:20:04,556 - INFO - Wish燕文专线平邮小包-特货 exported.
 2020-03-19 00:20:04,556 - INFO - WISH燕文C平邮小包 is ready to export.
 2020-03-19 00:20:04,655 - INFO - WISH燕文C平邮小包 exported.
 2020-03-19 00:20:04,656 - INFO - Wish燕邮宝平邮-特货 is ready to export.
 2020-03-19 00:20:04,717 - INFO - Wish燕邮宝平邮-特货 exported.
 2020-03-19 00:20:04,717 - INFO - Wish燕文航空挂号小包-普货 is ready to export.


Could not operate 0.35 with block values can't multiply sequence by non-int of type 'float'
  WISH燕文C平邮小包 Unnamed: 1 Unnamed: 2 Unnamed: 3  返回主目录
3          美国       7.68      74.05      53.39    NaN
4         加拿大       8.31      76.71      53.56    NaN
5          英国       8.33      74.14      52.74    NaN
6          法国       8.18       65.8       50.9    NaN
7          德国       8.26       70.5      49.38    NaN
  Wish燕邮宝平邮-特货 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4  返回主目录
4           美国         75        8.5         75        8.5    NaN
5           英国         78        6.5         58          7    NaN
6           法国         70          7         58          7    NaN
7           德国         74        6.5         52          7    NaN
8          西班牙         75          5         58          7    NaN
  Wish燕文航空挂号小包-普货 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5  \
4              美国         62         14         62         14         62   
5             加拿大         65        

 2020-03-19 00:20:04,864 - INFO - Wish燕文航空挂号小包-普货 exported.
 2020-03-19 00:20:04,865 - INFO - Wish燕文航空挂号小包-特货 is ready to export.
 2020-03-19 00:20:04,970 - INFO - Wish燕文航空挂号小包-特货 exported.
 2020-03-19 00:20:04,971 - INFO - WISH燕文C挂号小包 is ready to export.
 2020-03-19 00:20:05,112 - INFO - WISH燕文C挂号小包 exported.
 2020-03-19 00:20:05,113 - INFO - WISH燕文C优先小包 is ready to export.


  Wish燕文航空挂号小包-特货 Unnamed: 1 Unnamed: 2  返回主目录
4              美国         65       14.5    NaN
5             加拿大         65         19    NaN
6              英国         48         17    NaN
7              法国         68         21    NaN
8              德国         65         26    NaN
  WISH燕文C挂号小包 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5  \
4          美国       53.9       19.6         53       19.6         52   
5         加拿大       57.9         17       57.9         17       57.9   
6          英国         51       17.5         51       17.5         50   
7          法国         67         13       49.3       15.4       49.3   
8          德国       56.9       15.5         51       15.7       48.1   

  Unnamed: 6  返回主目录  
4       19.6    NaN  
5         17    NaN  
6       17.5    NaN  
7       15.4    NaN  
8       16.2    NaN  
  WISH燕文C优先小包 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5  返回主目录
3          美国       57.9       13.4       0.05      0.001          2    NaN
4

 2020-03-19 00:20:05,151 - INFO - WISH燕文C优先小包 exported.
 2020-03-19 00:20:05,152 - INFO - Wish燕邮宝挂号-特货 is ready to export.
 2020-03-19 00:20:05,212 - INFO - Wish燕邮宝挂号-特货 exported.
 2020-03-19 00:20:05,213 - INFO - Wish燕文航空易派小包-普货 is ready to export.
 2020-03-19 00:20:05,234 - INFO - Wish燕文航空易派小包-普货 exported.
 2020-03-19 00:20:05,236 - INFO - Wish燕文航空易派小包-特货 is ready to export.
 2020-03-19 00:20:05,305 - INFO - Wish燕文航空易派小包-特货 exported.
 2020-03-19 00:20:05,305 - INFO - Wish燕文专线追踪小包-普货 is ready to export.
 2020-03-19 00:20:05,332 - INFO - Wish燕文专线追踪小包-普货 exported.
 2020-03-19 00:20:05,332 - INFO - Wish燕文专线追踪小包-特货 is ready to export.
 2020-03-19 00:20:05,354 - INFO - Wish燕文专线追踪小包-特货 exported.
 2020-03-19 00:20:05,355 - INFO - WISH燕特快-普货 is ready to export.
 2020-03-19 00:20:05,381 - INFO - WISH燕特快-普货 exported.
 2020-03-19 00:20:05,382 - INFO - WISH燕特快-特货 is ready to export.
 2020-03-19 00:20:05,402 - INFO - WISH燕特快-特货 exported.


  Wish燕邮宝挂号-特货 Unnamed: 1 Unnamed: 2  返回主目录
4           美国         65         14    NaN
5           英国         48         17    NaN
6           法国         65         18    NaN
7           德国         60         17    NaN
8          意大利         65         23    NaN
  Wish燕文航空易派小包-普货 Unnamed: 1 Unnamed: 2    Unnamed: 3 Unnamed: 4  返回主目录
3              美国       65.5         14  0.001-2.0 KG       0.05    NaN
4             加拿大       62.5         18  0.001-2.0 KG       0.05    NaN
5              英国       34.5       16.5  0.001-2.0 KG       0.05    NaN
6              法国       52.5         17  0.001-2.0 KG       0.05    NaN
7              德国         37         21  0.001-2.0 KG       0.05    NaN
  Wish燕文航空易派小包-特货 Unnamed: 1 Unnamed: 2  返回主目录
4              美国         63         14    NaN
5             加拿大         57         21    NaN
6              英国         42       16.5    NaN
7              法国         61         17    NaN
8              德国         44         21    NaN
  Wish燕文专线追踪小包-普货 Unna